In [1]:
# %cd ..
!nvidia-smi

Mon Apr 13 21:51:21 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.64.00    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    25W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [2]:
!pip install gdown
!pip install tensorflow-gpu --user
!pip install --upgrade keras

     |████████████████████████████████| 421.8MB 35kB/s 
     |████████████████████████████████| 3.9MB 52.3MB/s 
     |████████████████████████████████| 450kB 57.9MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7540 sha256=3abad629aa5f8049f24fe2987e880f633106c42ae0a7a08a0978c2a8836d3420
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow 2.2.0rc2 has requirement gast==0.3.3, but you'll have gast 0.2.2 which is incompatible.
ERROR: tensorflow 2.2.0rc2 has requirement tensorboard<2.3.0,>=2.2.0, but you'll have tensorboard 2.1.1 which is incompatible.
ERROR: tensorflow 2.2.0rc2 has requirement tensorflow-estimator<2.3.0,>=2.2.0rc0, but you'll have tensorflow-estimator 2.1.0 which is incompatible.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer

In [3]:
%cd /content/drive/My\ Drive/person_detection/keras-retinanet
!pip install .
!python setup.py build_ext --inplace
%cd ../../../../sample_data/
!git clone https://github.com/Yannick947/WiderPerson.git
%cd ../../

/content/drive/My Drive/person_detection/keras-retinanet
Processing /content/drive/My Drive/person_detection/keras-retinanet
  Created wheel for keras-retinanet: filename=keras_retinanet-0.5.1-cp36-cp36m-linux_x86_64.whl size=172047 sha256=8722ca5a59db1513727945d765c1dbf3fe4ec6020a253c9955e0cdf16a2b0de6
  Stored in directory: /root/.cache/pip/wheels/ca/78/63/58275b4a0f024168bf718212448cbba87f45aa1948fdc1a291
  Created wheel for keras-resnet: filename=keras_resnet-0.1.0-py2.py3-none-any.whl size=13346 sha256=fd4682d2b4a67a79fc48aaeedb5f97f913696d9b34b8afe4b7bbde315b9dd8ef
  Stored in directory: /root/.cache/pip/wheels/80/dd/ac/842235b63dddac12faa4b48ebe58b8944e8c2e57c2e38dddb6
Successfully built keras-retinanet keras-resnet
running build_ext
skipping 'keras_retinanet/utils/compute_overlap.c' Cython extension (up-to-date)
copying build/lib.linux-x86_64-3.6/keras_retinanet/utils/compute_overlap.cpython-36m-x86_64-linux-gnu.so -> keras_retinanet/utils
/content/sample_data
Cloning into 'Wid

In [4]:
#quickfix error in callbacks of keras, pullrequest is already open in github
#later versions of tensorflow and keras should contain this fix

from google.colab import files
import shutil
%cd root/.local/lib/python3.6/site-packages/tensorflow_core/python/keras/
# files.download('callbacks.py')
!rm callbacks.py
shutil.copyfile('/content/drive/My Drive/person_detection/utils/callbacks.py', 
            './callbacks.py')
!ls
%cd ../../../../../../../

/root/.local/lib/python3.6/site-packages/tensorflow_core/python/keras
activations.py	   estimator			   optimizers.py
api		   initializers.py		   optimizer_v2
applications	   __init__.py			   premade
backend_config.py  keras_parameterized.py	   preprocessing
backend.py	   layers			   __pycache__
callbacks.py	   losses.py			   regularizers.py
callbacks_v1.py    metrics.py			   saving
constraints.py	   mixed_precision		   testing_utils.py
datasets	   models.py			   utils
distribute	   model_subclassing_test_util.py  wrappers
engine		   ops.py
/root


In [5]:
import numpy as np
import tensorflow as tf
import pandas as pd
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
from pandas.plotting import register_matplotlib_converters
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import TensorBoard
import urllib
import os
import csv
import cv2
import time
from PIL import Image
from IPython.core.debugger import set_trace
import shutil
from time import gmtime, strftime
# import TensorBoardColab
import pdb

from keras_retinanet import models
from keras_retinanet.utils.image import read_image_bgr, preprocess_image, resize_image
from keras_retinanet.utils.visualization import draw_box, draw_caption
from keras_retinanet.utils.colors import label_color


%matplotlib inline
%config InlineBackend.figure_format='retina'

register_matplotlib_converters()
sns.set(style='whitegrid', palette='muted', font_scale=1.5)

rcParams['figure.figsize'] = 22, 10

RANDOM_SEED = 42

np.random.seed(RANDOM_SEED)
tf.random.set_seed(RANDOM_SEED)


/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [0]:
annot_path = '/content/drive/My Drive/person_detection/WiderPerson/Annotations'
images_path = '/content/drive/My Drive/person_detection/WiderPerson/Images'
annot_csv_path = '/content/drive/My Drive/person_detection/keras-retinanet/annotations_new.csv'
keras_path = '/content/drive/My Drive/person_detection/keras-retinanet'
#Remove classes which shall be left out in the csv
classes_ids = {1:'pedestrian',
               2:'riders',
               3:'partially-visible',
               5:'crowd'}

# Training



In [0]:
PRETRAINED_RES50_VANILLA = './snapshots/resnet50_vanilla.h5'
PRETRAINED_RES152_VANILLA = './snapshots/resnet152_vanilla_oid.h5'
PRETRAINED_MODEL_WI = './snapshots/resnet50_csv_25.h5'
logdir = 'tensorboard/{}_1'.format(strftime("%Y-%m-%d-%H-%M-%S", gmtime()))
#hyperparams which will get logged for tensorboard use



In [0]:
%cd /content/drive/My Drive/person_detection/keras-retinanet

!python keras_retinanet/bin/train.py  --weights $PRETRAINED_RES152_VANILLA\
                                      --backbone 'resnet152'\
                                      --lr 1e-3\
                                      --batch-size 4\
                                      --steps 1000 \
                                      --epochs 40 \
                                      --weighted-average \
                                      --tensorboard-dir {logdir}\
                                      --filtered-above 0\
                                      --compute-val-loss\
                                      --random-transform\
                                      --num-classes 1\
                                      csv ../annotations_train.csv classes_filtered.csv \
                                      --val-annotations ../annotations_test.csv \
                                      # --config config.ini\
                                      # --random-transform \
%cd ../../../../../

/content/drive/My Drive/person_detection/keras-retinanet
Using TensorFlow backend.
2020-04-14 06:54:21.030368: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libnvinfer.so.6'; dlerror: libnvinfer.so.6: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2020-04-14 06:54:21.030455: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libnvinfer_plugin.so.6'; dlerror: libnvinfer_plugin.so.6: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2020-04-14 06:54:21.030469: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:30] Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.
../annotations_test.csv
Creating model, this may take a second...
2020-04-14 06:54:23.613717: W tensorflow/core/common_runtime

In [0]:
# use this code to prevent colab from disconnecting during longer training times in javascript console window
function ClickConnect(){
    console.log("Clicked on connect button"); 
    document.querySelector("colab-connect-button").click()
}
setInterval(ClickConnect,60000)

In [11]:
# memory footprint support libraries/code
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
 process = psutil.Process(os.getpid())
 print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
 print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm()

  Created wheel for gputil: filename=GPUtil-1.4.0-cp36-none-any.whl size=7413 sha256=8b3ea8291c98b34d74115dd9f565b5c52024da1617282ecb5d269fdf7baad4e6
  Stored in directory: /root/.cache/pip/wheels/3d/77/07/80562de4bb0786e5ea186911a2c831fdd0018bda69beab71fd
Successfully built gputil
Gen RAM Free: 26.0 GB  | Proc size: 506.5 MB
GPU RAM Free: 16280MB | Used: 0MB | Util   0% | Total 16280MB


## Debugging

In [0]:
%cd /content/drive/My Drive/person_detection/keras-retinanet
!python keras_retinanet/bin/debug.py --annotations \
                                     --anchors \
                                     --no-gui \
                                     csv annot_train.csv classes.csv
%cd ../../../../..

### Anchor Optimization

In [0]:
%cd /content/drive/My Drive/person_detection
!git clone https://github.com/martinzlocha/anchor-optimization.git
%cd anchor-optimization
!pip install .
!python setup.py build_ext --inplace

In [0]:
!pip install tensorflow-gpu==1.15
!python ./../anchor-optimization/optimize_anchors.py ./annot_train_filtered_600_2500.csv
!pip uninstall tensorflow-gpu==1.15

/content/drive/My Drive/person_detection/keras-retinanet


## Functions


In [0]:
# Generate the classes csv file
with open(annotations, newline='', mode='x') as csvfile:
  csv_writer = csv.writer(csvfile, delimiter=',')
  for filename in os.listdir(images_path)[0:10]:
    if str(filename + '.txt') in annot:
      f = open(annot_path + '/' +  filename + '.txt', 'r')
      
      for index, line in enumerate(f): 
        if index == 0: 
          if line.strip() == '0':
            print('Not any  object in the image!')
          continue
          
        else: 
          split_line = line[:line.find('/')].split(' ')
          first_char = split_line.pop(0)
          split_line.insert(len(split_line), first_char)
          split_line.insert(0, images_path + '/' + filename)
          #convert from index to class label 
          try:
            split_line[-1] = classes_ids[int(split_line[-1])]
          except: 
            continue
          split_line[0] = split_line[0]

          csv_writer.writerow(split_line)
      
      f.close()

#create indexing csv file
with open('classes.csv', newline='', mode='x') as csvfile:
  csv_writer = csv.writer(csvfile, delimiter=',')
  for index, key in enumerate(classes_ids.keys()):
    csv_writer.writerow([str(classes_ids[key]), index])


#Check for wrongly annotaded bounding boxes
path = '/content/drive/My Drive/person_detection/keras-retinanet/annotations.csv'
def check_bb(path):
  colnames = ['filename', 'x1', 'y1', 'x2', 'y2', 'class_label']
  df = pd.read_csv(path, names=colnames)
  df_new = df.loc[(df.x1 < df.x2) & (df.y1 < df.y2), : ]
  print ('Reduces shape from ', df.shape, 'to ', df_new.shape)
  return df_new





In [0]:
def replace_annoation_folder(new_folder, annot_path, old_img_path):
  '''Start this function from folder root, otherwise wont work properly'''

  df_annot = pd.read_csv(annot_path + '/annotations_pathkeras.csv',
                         names=['name', 'x1', 'y1', 'x2', 'y2', 'label'])
  df_annot = df_annot.reset_index().drop(0).drop(columns='index')
  df_annot['name'] = df_annot['name'].str.replace(old_img_path, new_folder)
  df_annot = df_annot.dropna()
  df_annot.to_csv(annot_path + '/annotations_new.csv',
                  index=None, header=None)
  return


old_folder = '/content/drive/My Drive/PersonDetection/WiderPerson/Images'
new_folder = '/content/drive/My Drive/person_detection/WiderPerson/Images'
annot_path_csv = '/content/drive/My Drive/person_detection/keras-retinanet'
replace_annoation_folder(new_folder, annot_path_csv, old_folder)
